In [1]:
import heapq

# 무한대 정의.
INF = float('inf')

# 최소신장트리 예제 입력.
verticies = [chr(i) for i in range(ord('A'), ord('G') + 1)]
weights = [
    [None,   29, None, None, None,   10, None],
    [  29, None,   16, None, None, None,   15],
    [None,   16, None,   12, None, None, None],
    [None, None,   12, None,   22, None,   18],
    [None, None, None,   22, None,   27,   25],
    [  10, None, None, None,   27, None, None],
    [None,   15, None,   18,   25, None, None]
]

# 허프만 코드 예제 입력.
labels = [chr(i) for i in range(ord('A'), ord('H') + 1)]
frequencies = [24, 3, 8, 10, 33, 6, 4, 12]

# 그래프 색칠 예제 입력.
states = ['NT', 'WA', 'Q', 'SA', 'NSW', 'V']
color_names = ['None', 'RED', 'GREEN', 'BLUE', 'YELLOW', 'PURPLE']
graph = [
    [0, 1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 1, 1, 0],
    [1, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0]
]

def main():
    # Prim의 최소신장트리 알고리즘.
    print('# Prim의 최소신장트리 알고리즘.')

    mst_prim(verticies, weights)
    print()

    # Kruskal의 최소신장트리 알고리즘.
    print('# Kruskal의 최소신장트리 알고리즘.')

    mst_kruskal(verticies, weights)
    print()

    # 허프만 코드 트리 알고리즘.
    print('# 허프만 코드 트리 알고리즘.')

    huffman_code_tree = generate_huffman_code_tree(frequencies, labels)
    print(f'Root node of the huffman code tree: {huffman_code_tree[0]}\n')

    # N-Queen 문제 알고리즘.
    print('# N-Queen 문제 알고리즘.')

    for n in range(5):
        print(f'Solution to N-Queen Problem when n is {n}')
        solve_n_queen_problem([[0 for _ in range(n)] for _ in range(n)], 0)

    # 그래프 색칠 문제 알고리즘.
    print('# 그래프 색칠 문제 알고리즘.')

    print('3 Colors:')
    graph_coloring(graph, 3, states)
    print()

    print('2 Colors:')
    graph_coloring(graph, 2, states)
    print()

    print('4 Colors:')
    graph_coloring(graph, 4, states)

In [2]:
# Prim의 최소신장트리 알고리즘.
def mst_prim(verticies, adjacent):
    vertex_count = len(verticies)
    dist = [INF for _ in range(vertex_count)]         # 각 정점으로부터 최소한의 거리를 저장할 리스트. 최초에는 모두 INF로 초기화해준다.
    selected = [False for _ in range(vertex_count)]   # MST에 특정 정점이 포함되었는가를 저장하는 리스트.
    
    dist[0] = 0     # 최초 정점으로 인덱스가 0인 정점을 선택.

    # 정점의 개수만큼 반복.
    for _ in range(vertex_count):
        current_vertex_index = get_min_vertex(dist, selected)   # 현재 MST에 포함할 정점을 선택.
        selected[current_vertex_index] = True                   # 정점 선택 후 selected에 추가.

        # 다음 정점을 위한 작업.
        # 현재 정점과 이어진 정점 중 가중치가 가장 낮은 정점을 선택할 수 있도록 dist 갱신.
        for next_vertex_index in range(vertex_count):
            is_selected = selected[next_vertex_index]                                                                    # 이미 선택된 정점인가.
            is_connected = adjacent[current_vertex_index][next_vertex_index] is not None                                 # 현재 정점과 연결된 정점인가.
            is_closer = is_connected and adjacent[current_vertex_index][next_vertex_index] < dist[next_vertex_index]     # 더 가까운 정점인가.

            if not is_selected and is_closer:
                dist[next_vertex_index] = adjacent[current_vertex_index][next_vertex_index]

        print(f'{verticies[current_vertex_index]}: {dist}')     # 선택된 정점과 그에 따른 dist 출력.

# 현재 정점과 이어진 정점 중 가장 가중치가 낮은 정점 찾기.
def get_min_vertex(dist, selected):
    min_vertex_index = -1       # 아직 인덱스가 정해지지 않았으므로 -1로 초기값 설정.
    min_dist = INF              # 아직 최소 거리가 정해지지 않았으므로 INF로 초기값 설정.

    # dist 내의 정점에 대해서 반복.
    for v_index in range(len(dist)):
        # 현재 선택된 정점보다 거리가 작으면 min_vertex_index 및 min_dist 갱신.
        if not selected[v_index] and dist[v_index] < min_dist: 
            min_dist = dist[v_index]
            min_vertex_index = v_index

    return min_vertex_index

In [3]:
# Kruskal의 최소신장트리 알고리즘.
def mst_kruskal(verticies, adjacent):
    vertex_count = len(verticies)
    dsets = disjointSet(vertex_count)     # 사이클 확인을 위한 서로소 집합 생성.
    possible_edges = []                   # 정점 간 가능한 간선을 저장할 리스트.

    # 가능한 모든 간선에 대해 해당 간선이 실제로 존재하면 possible_edges에 추가.
    for i in range(vertex_count):
        for j in range(i + 1, vertex_count):
            if adjacent[i][j] is not None:
                possible_edges.append((i, j, adjacent[i][j]))

    possible_edges.sort(key=lambda x: x[2])     # 간선 간 가중치에 대해 오름차순 정렬.

    edge_count = 0

    # 가능한 간선에 대해서 MST 생성.
    for edge in possible_edges:
        begin_vertex_parent_index = dsets.find(edge[0])
        end_vertex_parent_index = dsets.find(edge[1])

        # 시점과 종점이 서로 다른 집합에 속해 있으면 사이클이 생기지 않으므로 이를 MST에 추가.
        if begin_vertex_parent_index != end_vertex_parent_index:
            dsets.union(begin_vertex_parent_index, end_vertex_parent_index)

            v1 = verticies[edge[0]]
            v2 = verticies[edge[1]]
            weight = edge[2]

            print(f'Edge added: between {v1} and {v2} (weight: {weight})')

            # 모든 정점이 MST에 속하게 되면 반복 종료.
            edge_count += 1
            if edge_count == vertex_count - 1:
                break

class disjointSet:
    def __init__(self, n):
        # 리스트의 인덱스 자체가 요소가 됨. 해당 인덱스의 리스트 값은 요소의 부모의 인덱스가 됨. -1일 경우 자기 자신이 루트 노드. (또는 부모 노드)
        self.parents = [-1 for _ in range(n)]

        # 서로 다른 집합의 개수.
        self.set_count = n

    # 특정 원소가 속한 집합의 루트 노드의 인덱스 반환.
    def find(self, vertex_index):
        # 자기 자신이 루트 노드가 될 때까지 반복.
        while self.parents[vertex_index] >= 0:
            vertex_index = self.parents[vertex_index]

        return vertex_index

    # 특정 원소를 다른 원소의 자식 노드로서 병합.
    def union(self, s1, s2):
        self.parents[s1] = s2       # s1의 부모 노드를 s2로 갱신.
        self.set_count -= 1         # 두 집합을 병합하므로 집합 개수 감소.

    # 집합으로서 반환.
    def get_set(self):
        ret = [set() for _ in range(len(self.parents))]

        for index, parent in enumerate(self.parents):
            if parent == -1:
                ret[index].add(index)
            else:
                parent_index = self.find(index)
                while self.parents[index] >= 0:
                    ret[parent_index].add(index)
                    index = self.parents[index]

        for set_ in ret:
            if len(set_) == 0:
                ret.remove(set_)

        return ret

In [4]:
# 허프만 코드 트리 알고리즘.
def generate_huffman_code_tree(frequencies, labels):
    label_count = len(labels)
    heap = [(frequencies[i], labels[i]) for i in range(label_count)]    # 빈도수와 해당 빈도수에 대응하는 문자를 튜플로 엮어 리스트 생성.
    heapq.heapify(heap)                                                 # 위에서 생성한 리스트를 최소 힙으로 변환.

    # n개의 노드 병합 시 총 n - 1번의 병합이 요구됨.
    for _ in range(0, label_count - 1):
        # 가장 빈도수가 작은 두 노드 추출.
        least_freq_1 = heapq.heappop(heap)
        least_freq_2 = heapq.heappop(heap)

        # 추출한 두 노드를 병합.
        heapq.heappush(heap, (least_freq_1[0] + least_freq_2[0], least_freq_1[1] + least_freq_2[1]))
        print(f'{least_freq_1} + {least_freq_2}')

    return heap

In [5]:
# N-Queen 문제 알고리즘.
def solve_n_queen_problem(board, row):
    # 현재 행이 보드의 행의 수와 같다면 모든 행에 대해서 이미 수행한 것이므로 결과를 출력.
    if len(board) == row:
        for row in board:
            print(row)
        print()
        return

    # 현재 행의 열에 대해서 여왕을 둘 수 있으면 여왕을 두는 경우를 고려.
    for col in range(len(board[row])):
        if is_safe_n_queen(board, row, col):
            board[row][col] = 1     # 여왕 배치.
            solve_n_queen_problem(board, row + 1)
            board[row][col] = 0     # 해당 위치에 대해 살펴봤으므로 회수.

# 해당 위치에 여왕을 둘 수 있는가.
def is_safe_n_queen(board, row, col):
    # 수직 방향 검사.
    for r in range(row):
        if board[r][col] == 1:
            return False

    # 내려가는 대각선 검사.
    for r, c in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if board[r][c] == 1:
            return False

    # 올라가는 대각선 검사.
    for r, c in zip(range(row - 1, -1, -1), range(col + 1, len(board))):
        if board[r][c] == 1:
            return False

    return True

In [6]:
# 그래프 색칠 알고리즘.
def graph_coloring(graph, color_count, states):
    vertex_colors = [0 for _ in range(len(graph))]      # 그래프에 칠해진 색상 정의. (0은 None, 아무 것도 칠해지지 않은 상태)
    
    # 그래프 색칠에 성공하면 결과 출력.
    if graph_coloring_dfs(graph, vertex_colors, 0, color_count):
        for i in range(len(graph)):
            print(f'{states[i]:3s}: {color_names[vertex_colors[i]]}')
    else:
        print('Graph can not be colored.')

def graph_coloring_dfs(graph, vertex_colors, vertex_index, color_count):
    # 현재까지 색칠한 정점의 개수가 그래프 내의 정점의 개수가 되면 참 반환.
    if vertex_index == len(graph):
        return True

    # 색칠 가능한 모든 색상에 대해 반복.
    for color_index in range(1, color_count + 1):
        # 현재 색상으로 색칠이 가능하면 해당 색을 칠한 경우로 재귀 진행.
        if is_safe_graph_coloring(graph, vertex_colors, vertex_index, color_index):
            vertex_colors[vertex_index] = color_index

            # 현재 색상으로 색칠했을 때 모든 정점을 색칠할 수 있으면 참 반환.
            if graph_coloring_dfs(graph, vertex_colors, vertex_index + 1, color_count):
                return True

            # 현재 색상으로 색칠했을 때 모든 정점을 색칠할 수 없으면 색칠 취소.
            vertex_colors[vertex_index] = 0

    # 색칠 가능한 경우가 없으면 거짓 반환.
    return False

def is_safe_graph_coloring(graph, vertex_colors, vertex_index, color_index):
    # 그래프에 있는 모든 정점에 대해 반복.
    for i in range(len(graph[vertex_index])):
        # 현재 정점과 맞닿은 정점의 색이 칠하고자 하는 색과 동일하면 거짓 반환.
        if graph[vertex_index][i] == 1 and vertex_colors[i] == color_index:
            return False

    return True

In [7]:
if __name__ == '__main__':
    main()

# Prim의 최소신장트리 알고리즘.
A: [0, 29, inf, inf, inf, 10, inf]
F: [0, 29, inf, inf, 27, 10, inf]
E: [0, 29, inf, 22, 27, 10, 25]
D: [0, 29, 12, 22, 27, 10, 18]
C: [0, 16, 12, 22, 27, 10, 18]
B: [0, 16, 12, 22, 27, 10, 15]
G: [0, 16, 12, 22, 27, 10, 15]

# Kruskal의 최소신장트리 알고리즘.
Edge added: between A and F (weight: 10)
Edge added: between C and D (weight: 12)
Edge added: between B and G (weight: 15)
Edge added: between B and C (weight: 16)
Edge added: between D and E (weight: 22)
Edge added: between E and F (weight: 27)

# 허프만 코드 트리 알고리즘.
(3, 'B') + (4, 'G')
(6, 'F') + (7, 'BG')
(8, 'C') + (10, 'D')
(12, 'H') + (13, 'FBG')
(18, 'CD') + (24, 'A')
(25, 'HFBG') + (33, 'E')
(42, 'CDA') + (58, 'HFBGE')
Root node of the huffman code tree: (100, 'CDAHFBGE')

# N-Queen 문제 알고리즘.
Solution to N-Queen Problem when n is 0

Solution to N-Queen Problem when n is 1
[1]

Solution to N-Queen Problem when n is 2
Solution to N-Queen Problem when n is 3
Solution to N-Queen Problem when n is 4
[0, 1, 0, 0]
[0, 0, 0,